In [1]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 68.5 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 67.1 MB/s 
     |████████████████████████████████| 212 kB 51.5 MB/s 
     |████████████████████████████████| 127 kB 61.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 5.2 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.utils import shuffle
import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/ERAI Dataset_with_translation


In [5]:
pd.set_option('display.max_rows', None)

In [6]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('ERAI_Dataset_pairwise_test_with_translation.json')
df.reset_index(drop = True,inplace = True)
df = df.dropna(axis = 0)



df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))

df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2_']=df['post2'].apply(lambda x:x.replace('\n',''))


In [7]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [8]:

def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.float),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.float),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [9]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes_cls,n_classes_reg):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.1)
    self.out_cls = nn.Linear(self.bert.config.hidden_size, n_classes_cls)
   

    self.out_reg = nn.Linear(self.bert.config.hidden_size, n_classes_reg)
    self.sigmoid = nn.Sigmoid()

  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    cls_output = []
    sent1_output = []
    sent2_output = []
    

    for last_output,len1,len2 in zip(output[0],length1,length2):
      
      
      sent1_output.append(torch.mean(last_output[1:(len1+1)],dim=0))
      sent2_output.append(torch.mean(last_output[len1+2:(len1+2+len2)],dim=0))

    cls_output = output[1]
    sent1_output = torch.stack(sent1_output)
    
    sent2_output = torch.stack(sent2_output)
    
    cls_output = self.sigmoid(self.out_cls(cls_output))
    
    
    
    
    return cls_output,self.out_reg(sent1_output),self.out_reg(sent2_output)

In [10]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.mpp_label)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [11]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [12]:
EPOCHS = 30


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_reg = nn.MSELoss().to(device)
loss_cls = nn.BCELoss().to(device)






In [13]:
def test_prediction(model, 
  test_dataloader, 
  optimizer,
  device,
  max_len
  ):
   
  first_sen_mpp = []
  first_sen_ml = []
  second_sen_mpp = []
  second_sen_ml = []

  first_real_mpp = []
  first_real_ml = []
  second_real_mpp = []
  second_real_ml = []

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)

      output_cls,output_sen1,output_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )

      
      for i in output_sen1:
        first_sen_mpp.append(i[0].cpu().item())
        first_sen_ml.append(i[1].cpu().item())
      for i in output_sen2:
        second_sen_mpp.append(i[0].cpu().item())
        second_sen_ml.append(i[1].cpu().item())
      for i in first_targets:
        first_real_mpp.append(i[0].cpu().item())
        first_real_ml.append(i[1].cpu().item())
      for i in second_targets:
        second_real_mpp.append(i[0].cpu().item())
        second_real_ml.append(i[1].cpu().item())
  test_pred_mpp = first_sen_mpp+second_sen_mpp
  test_pred_ml = first_sen_ml+second_sen_ml
  real_test_mpp = first_real_mpp+second_real_mpp
  real_test_ml = first_real_ml+second_real_ml


  return test_pred_mpp,test_pred_ml,real_test_mpp,real_test_ml

In [14]:

def train_and_eval(model, 
  train_loader, 
  validation_loader, 
  optimizer,  
  device,
  epoch,
  test_dataloader,
  real_top_mpp,
  real_top_ml
  ):
  model.train()
  max_len = 256
  average_state = 10000
  for epo in range(epoch):
    
    train_loss = []
    
    global train_mpp_label
    train_mpp_label = []
    global train_ml_label
    train_ml_label = []

    global real_train_mpp 
    real_train_mpp = []
    global real_train_ml
    real_train_ml = []
    

    validation_loss = []
    
    validation_mpp_predictions = []
    validation_ml_predictions = []


    

    real_validation_mpp = []
    real_validation_ml = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      output_cls,output_sen1,output_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      
      
      mpp_ml_label = torch.stack((mpp_targets,ml_targets),1)
      
      loss1 = loss_cls(output_cls.view(-1),mpp_ml_label.view(-1)).to(device)
      
      loss2 = loss_reg(output_sen1.view(-1),first_targets.view(-1))
      
      loss3 = loss_reg(output_sen2.view(-1),second_targets.view(-1))
      
      loss = loss1+10*loss2+10*loss3
 

      train_loss.append(loss.item())
     

    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    

    mpp_pred_value,ml_pred_value,real_mpp_label,real_ml_label = test_prediction(model, 
      test_dataloader,  
      optimizer,
      device,
      max_len
      )

    pred_top_mpp = []
    pred_top_ml = []

    final_train_loss = np.mean(train_loss)



    real_mmp_average = np.mean(real_top_mpp[0:8])
    real_ml_average = np.mean(real_top_ml[0:8])

    
    pred_mmp_average = []
    pred_ml_average = []
    pred_mmp_rank = np.argsort(mpp_pred_value)[::-1][0:8]
    pred_ml_rank = np.argsort(ml_pred_value)[0:8]



    for i,j in zip(pred_mmp_rank,pred_ml_rank):
      pred_mmp_average.append(real_mpp_label[i])
      pred_ml_average.append(real_ml_label[j])

    pred_mmp_average = np.mean(pred_mmp_average)
    pred_ml_average = np.mean(pred_ml_average)

    mmp_difference = pred_mmp_average-real_mmp_average
    ml_difference = real_ml_average-pred_ml_average

    if (mmp_difference+ml_difference)/2 < average_state:
      accuracy_state = (mmp_difference+ml_difference)/2
      torch.save(model.state_dict(), 'task2_regression.bin')




    print(
'Epoch [{}] Train Totol Loss: {:.4f}, REAL AVERAGE MPP: {:.4F}, REAL AVERAGE ML: {:.4F}, PRED AVERAGE MPP: {:.4F} PRED AVERAGE ML: {:.4F}'
    .format(epo+1,final_train_loss,real_mmp_average,real_ml_average,pred_mmp_average,pred_ml_average))

In [15]:
random_seed = [171,354,550,667,985]
model_name = ['roberta','chi','fin','sec']

for i in model_name:
  if i == 'chi':
    PRETRAINED_MODEL_NAME = "bert-base-chinese"
    tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'roberta':
    PRETRAINED_MODEL_NAME = '/content/drive/MyDrive/ROBERT_4_model.bin'
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
  if i == 'fin':
    PRETRAINED_MODEL_NAME = 'yiyanghkust/finbert-tone'
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'sec':
    PRETRAINED_MODEL_NAME = "nlpaueb/sec-bert-shape"
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)



  for j in random_seed:
    print(i)
    print(j)
    df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
    
    df = df.dropna(axis = 0)

    df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
    df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))

    df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
    df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))
    
    df = shuffle(df,random_state=j)

    train = df[0:156]

    test = df[156:200]

    if i=='chi'or i == 'roberta':
      train_post1 = train['post1']
      train_post2 = train['post2']
      test_post1 = test['post1']
      test_post2 = test['post2']
    else:
      
      train_post1 = train['post1_en']
      train_post2 = train['post2_en']
      test_post1 = test['post1_en']
      test_post2 = test['post2_en']

    train_mpp_label = train['MPP_label']
    train_ml_label = train['ML_label']
    train_mpp_value1 = train['MPP1'] 
    train_mpp_value2 = train['MPP2']
    train_ml_value1 = train['ML1']
    train_ml_value2 = train['ML2']


    
    test_mpp_label = test['MPP_label']
    test_ml_label = test['ML_label']
    test_mpp_value1 = test['MPP1'] 
    test_mpp_value2 = test['MPP2']
    test_ml_value1 = test['ML1']
    test_ml_value2 = test['ML2']

    all_test_mpp = pd.concat([test_mpp_value1,test_mpp_value2],axis=0)
    all_test_ml = pd.concat([test_ml_value1,test_ml_value2],axis=0)

    real_mpp_rank = np.argsort(all_test_mpp.to_list())
    real_mpp_rank= real_mpp_rank[::-1]
    real_ml_rank = np.argsort(all_test_ml.to_list())

    real_top_mpp = []
    real_top_ml = []

    for i,j in zip(real_mpp_rank,real_ml_rank):
        
      real_top_mpp.append(all_test_mpp.tolist()[i])
      real_top_ml.append(all_test_ml.tolist()[j])

    max_len = 256
    batch_size = 16
    train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
    test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)
    label_num_cls = 2
    label_num_reg = 2
    model= BertNerClassifier(label_num_cls,label_num_reg).to(device)
    optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)

    train_and_eval(model, 
    train_dataloader, 
    test_dataloader, 
    optimizer,  
    device,
    EPOCHS,
    test_dataloader,
    real_top_mpp,
    real_top_ml
    )
    

    
    
    
  


roberta
171


Some weights of the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin and are newly initialized: ['bert.pooler.

Epoch [1] Train Totol Loss: 1.0575, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0314 PRED AVERAGE ML: -0.0161
Epoch [2] Train Totol Loss: 0.8226, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0244 PRED AVERAGE ML: -0.0106
Epoch [3] Train Totol Loss: 0.8074, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0253 PRED AVERAGE ML: -0.0117
Epoch [4] Train Totol Loss: 0.8394, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0313 PRED AVERAGE ML: -0.0117
Epoch [5] Train Totol Loss: 0.8419, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0371 PRED AVERAGE ML: -0.0117
Epoch [6] Train Totol Loss: 0.8176, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0268 PRED AVERAGE ML: -0.0204
Epoch [7] Train Totol Loss: 0.7245, REAL AVERAGE MPP: 0.0807, REAL AVERAGE ML: -0.0640, PRED AVERAGE MPP: 0.0253 PRED AVERAGE ML: -0.0117
Epoch [8] Train Totol Loss: 0.7044

Some weights of the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin and are newly initialized: ['bert.pooler.

Epoch [1] Train Totol Loss: 1.1412, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0582 PRED AVERAGE ML: -0.0132
Epoch [2] Train Totol Loss: 0.8556, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0475 PRED AVERAGE ML: -0.0166
Epoch [3] Train Totol Loss: 0.7768, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0475 PRED AVERAGE ML: -0.0166
Epoch [4] Train Totol Loss: 0.8139, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0468 PRED AVERAGE ML: -0.0140
Epoch [5] Train Totol Loss: 0.8045, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0468 PRED AVERAGE ML: -0.0141
Epoch [6] Train Totol Loss: 0.7469, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0468 PRED AVERAGE ML: -0.0141
Epoch [7] Train Totol Loss: 0.7122, REAL AVERAGE MPP: 0.1020, REAL AVERAGE ML: -0.0762, PRED AVERAGE MPP: 0.0468 PRED AVERAGE ML: -0.0141
Epoch [8] Train Totol Loss: 0.6945

...

Epoch [1] Train Totol Loss: 1.7830, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0166 PRED AVERAGE ML: -0.0391
Epoch [2] Train Totol Loss: 1.0531, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0184 PRED AVERAGE ML: -0.0180
Epoch [3] Train Totol Loss: 0.8005, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0192 PRED AVERAGE ML: -0.0106
Epoch [4] Train Totol Loss: 0.7465, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0192 PRED AVERAGE ML: -0.0106
Epoch [5] Train Totol Loss: 0.6986, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0186 PRED AVERAGE ML: -0.0106
Epoch [6] Train Totol Loss: 0.6921, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0186 PRED AVERAGE ML: -0.0186
Epoch [7] Train Totol Loss: 0.6728, REAL AVERAGE MPP: 0.0728, REAL AVERAGE ML: -0.0799, PRED AVERAGE MPP: 0.0173 PRED AVERAGE ML: -0.0186
Epoch [8] Train Totol Loss: 0.6642

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 1.2390, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0108 PRED AVERAGE ML: -0.0278
Epoch [2] Train Totol Loss: 0.8895, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0098 PRED AVERAGE ML: -0.0327
Epoch [3] Train Totol Loss: 0.8286, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0100 PRED AVERAGE ML: -0.0352
Epoch [4] Train Totol Loss: 0.7276, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0094 PRED AVERAGE ML: -0.0352
Epoch [5] Train Totol Loss: 0.6922, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0094 PRED AVERAGE ML: -0.0352
Epoch [6] Train Totol Loss: 0.6848, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0094 PRED AVERAGE ML: -0.0337
Epoch [7] Train Totol Loss: 0.6689, REAL AVERAGE MPP: 0.0869, REAL AVERAGE ML: -0.0781, PRED AVERAGE MPP: 0.0094 PRED AVERAGE ML: -0.0352
Epoch [8] Train Totol Loss: 0.6626

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 1.6559, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0395 PRED AVERAGE ML: -0.0107
Epoch [2] Train Totol Loss: 0.9763, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0386 PRED AVERAGE ML: -0.0058
Epoch [3] Train Totol Loss: 0.8305, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0384 PRED AVERAGE ML: -0.0037
Epoch [4] Train Totol Loss: 0.7362, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0384 PRED AVERAGE ML: -0.0086
Epoch [5] Train Totol Loss: 0.7087, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0384 PRED AVERAGE ML: -0.0086
Epoch [6] Train Totol Loss: 0.6900, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0384 PRED AVERAGE ML: -0.0086
Epoch [7] Train Totol Loss: 0.6775, REAL AVERAGE MPP: 0.1100, REAL AVERAGE ML: -0.0797, PRED AVERAGE MPP: 0.0434 PRED AVERAGE ML: -0.0086
Epoch [8] Train Totol Loss: 0.6686

In [16]:
model.load_state_dict(torch.load('double_value_regression_withcls.bin', map_location=torch.device("cuda:0")))

FileNotFoundError: ignored

In [ ]:
def generate_test_data(post1,post2,max_len):
 
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

In [ ]:
 
class testSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.first_sen_len[index],self.second_sen_len[index]

In [ ]:
def generate_test_dataloader(post1,post2,max_len,batch_size):
  data = generate_test_data(post1,post2,max_len)
  data = testSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader

In [ ]:
test_post1 = df2['post1']
test_post2 = df2['post2']
test_loader = generate_test_dataloader(test_post1,test_post2,256,1)

In [ ]:
def real_prediction(model, 
  test_dataloader,  
  optimizer,
  device
  ):
  
  test_mpp_label = []
  test_ml_label = []


  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        
        first_sen_len = data[3].to(device)
        second_sen_len = data[4].to(device)
      

        output_cls,output_sen1,output_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
     
     
        for i, j in zip(output_sen1,output_sen2):
          if i[0]>j[0]:
            test_mpp_label.append(1)
          else:
            test_mpp_label.append(0)
          if i[1]>j[1]:
            test_ml_label.append(0)
          else:
            test_ml_label.append(1)
        
  return test_mpp_label,test_ml_label
      



In [ ]:
test_mpp_predictions,test_ml_predictions = real_prediction(model,test_loader,optimizer,device)

In [ ]:
df2['MPP_prediction'] = test_mpp_predictions
df2['ML_prediction'] = test_ml_predictions
df2.to_json('ERAI_pairwise_[aiml]_[1].json')

In [ ]:
df2